In [2]:
import os
import pandas as pd

# ==========================================
# 1. 설정
# ==========================================
DIR_CLEAN = "BOK_Cleaned"
DIR_TOKEN = "BOK_Tokenized"
SCORE_FILE = "BOK 회의록 점수 및 토큰.csv"
# ==========================================

print(f">>> 엑셀로 저장 하는 중.\n")

if not os.path.exists(SCORE_FILE):
    print(f"오류: '{SCORE_FILE}' 파일이 없습니다.")
else:
    df_scores = pd.read_csv(SCORE_FILE)
    final_data = []

    for index, row in df_scores.iterrows():
        # 1. 날짜 정리
        raw_date = str(row['날짜'])
        target_filename_date = raw_date.replace(".", "").replace("-", "").strip()
        
        try:
            ts = pd.to_datetime(raw_date)
            target_filename_date = ts.strftime("%Y%m%d")
        except:
            pass

        filename = target_filename_date + ".txt"
        
        # 2. 본문 가져오기
        content_text = ""
        path_clean = os.path.join(DIR_CLEAN, filename)
        if os.path.exists(path_clean):
            with open(path_clean, "r", encoding="utf-8") as f:
                content_text = f.read().replace("\n", " ").replace("\r", "")
        
        # 3. 토큰 가져오기
        token_text = ""
        path_token = os.path.join(DIR_TOKEN, filename)
        if os.path.exists(path_token):
            with open(path_token, "r", encoding="utf-8") as f:
                token_text = f.read().replace("\n", " ")
        
        # 4. 데이터 합치기
        final_data.append({
            "date": raw_date,
            "content": content_text,
            "tokens": token_text,
            "ngram": row['근거_단어'],
            "score": row['어조점수']
        })

    # 5. 엑셀 파일로 저장 (가장 안전함!)
    df_final = pd.DataFrame(final_data)
    df_final = df_final[['date', 'content', 'tokens', 'ngram', 'score']]
    
    save_name = "BOK compile.xlsx"
    
    try:
        df_final.to_excel(save_name, index=False, engine='openpyxl')
        print(f"\n>>> 성공. '{save_name}' 파일이 생성되었습니다.")
        
    except ImportError:
        print("\n엑셀 저장 기능 누락.")
    except Exception as e:
        print(f"\n엑셀 저장 실패 ({e}). 텍스트 파일로 저장.")
        df_final.to_csv("BOK compile.txt", sep='\t', index=False, encoding="utf-8-sig")

>>> 엑셀로 저장 하는 중.


>>> 성공. 'BOK compile.xlsx' 파일이 생성되었습니다.
